In [17]:
import pandas as pd
from datetime import datetime
import numpy as np

In [65]:
def convert_values(value):
    if value == 'NA':
        return 0
    else:
        return value

# Data load
## listings

In [100]:
listings_data_types = {'id':'int64', 'host_id':'int32', \
       'latitude':'float32', \
       'longitude':'float32', 'accommodates':np.ubyte,  \
       'availability_30':np.ubyte, 'availability_60':np.ubyte, 'availability_90':np.ubyte, 'availability_365':np.ushort,  'number_of_reviews':np.ushort, \
       'number_of_reviews_ltm':np.ushort, 'number_of_reviews_l30d':np.ubyte,  \
       'review_scores_rating':'float32', 'review_scores_accuracy':'float32', \
       'review_scores_cleanliness':'float32', 'review_scores_checkin':'float32', 'review_scores_communication':'float32', 'review_scores_location':'float32', \
       'review_scores_value':'float32', 'calculated_host_listings_count':np.ushort, \
       'calculated_host_listings_count_entire_homes':np.ushort, 'calculated_host_listings_count_private_rooms':np.ushort, \
        'calculated_host_listings_count_shared_rooms':np.ubyte, 'reviews_per_month':'float32'}

listings = pd.read_csv("http://data.insideairbnb.com/united-kingdom/england/london/2023-09-06/data/listings.csv.gz", \
                       low_memory=False, \
                       verbose=True, \
                       converters={'bedrooms': convert_values, 'beds': convert_values, 'maximum_nights':convert_values}, \
                       dtype = listings_data_types, \
                       parse_dates=['last_scraped', 'host_since','calendar_updated','calendar_last_scraped', 'first_review', 'last_review'])

Tokenization took: 2163.43 ms
Type conversion took: 1564.24 ms
Parser memory cleanup took: 13.16 ms


## calendar

In [125]:
calendar_data_types = {'listing_id': 'int64', \
    'minimum_nights': np.single, \
    'maximum_nights':'float32'}


calendar = pd.read_csv("http://data.insideairbnb.com/united-kingdom/england/london/2023-09-06/data/calendar.csv.gz", \
                       low_memory=False, \
                       verbose=True, \
                       converters={'adjusted_price': convert_values}, \
                       dtype = calendar_data_types, \
                       parse_dates=['date'])


Tokenization took: 12233.06 ms
Type conversion took: 9556.93 ms
Parser memory cleanup took: 237.70 ms


## reviews

In [134]:
reviews = pd.read_csv("http://data.insideairbnb.com/united-kingdom/england/london/2023-09-06/data/reviews.csv.gz", \
                      low_memory=False, \
                      verbose=True, \
                      converters={'adjusted_price': convert_values}, \
                      parse_dates=['date'])

Tokenization took: 5540.43 ms
Type conversion took: 3139.70 ms
Parser memory cleanup took: 33.45 ms


# memory optimization
- A regular read_csv for listings implies a memory usage of   50.32MB, but using dtype=listings_data_types   40.09MB:  18.7% reduction

- A regulat read_csv for calendar implies a memory usage of 1714.34MB, but using dtype=calendar_data_types 1469.43MB:  14.3% reduction

  A regulat read_csv for reviews  implies a memory usage of   72.37MB, but using dtype=reviews_data_types ther is not memory save.

I got this figures with 'sum(listings.memory_usage())/1024/1024'

# Ten questions

In [103]:
pd.options.display.max_columns = calendar.shape[1]
calendar.describe(include="all")


,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
count,3.210036e+07,32100363,32100363,32100016,32100016,3.209994e+07,3.209994e+07
unique,NaN,366,2,4485,4482,NaN,NaN
top,NaN,2024-03-07,f,$100.00,$100.00,NaN,NaN
freq,NaN,87946,21434951,892273,882212,NaN,NaN
mean,3.586456e+17,NaN,NaN,NaN,NaN,7.855209e+00,4.952079e+05
std,4.121105e+17,NaN,NaN,NaN,NaN,3.256786e+01,3.242923e+07
min,1.391300e+04,NaN,NaN,NaN,NaN,1.000000e+00,1.000000e+00
25%,2.215800e+07,NaN,NaN,NaN,NaN,1.000000e+00,1.000000e+02
50%,4.935862e+07,NaN,NaN,NaN,NaN,2.000000e+00,5.000000e+02
75%,8.164791e+17,NaN,NaN,NaN,NaN,4.000000e+00,1.125000e+03


## 1.- For the compliance department ==> Which percentage of the listings have informed license?

In [190]:
total_listings = listings.shape[0]
grouped= listings.groupby(["license"],group_keys=False)
licenses = grouped.size()
print("Only {:5.4f}% of listings have reported license ".format(100* licenses.iloc[0]/total_listings))

Only 0.0011% of listings have reported license 


## 2.- For the compliance department ==> Which percentage of the listings have a verified host?

In [176]:
total_listings = listings.shape[0]
grouped= listings.groupby(["host_identity_verified"],group_keys=False)
verifications = grouped.size()
print("Only {:5.4f}% of listings have a verified host ".format(100* verifications.iloc[0]/total_listings))

Only 12.6771% of listings has a verified host 


## 3.- For xxxxxx ==> ??????

In [201]:
grouped= calendar.groupby(["listing_id", "available"],group_keys=False)

In [204]:
booked = grouped.size().unstack()
booked.fillna(0)
booked["booked"]= booked["f"]/365
booked

available,f,t,booked
listing_id,,,
13913,5.0,360.0,0.013699
15400,292.0,73.0,0.800000
17402,65.0,300.0,0.178082
24328,365.0,NaN,1.000000
25123,365.0,NaN,1.000000
...,...,...,...
973781286754517228,126.0,239.0,0.345205
973801695874775338,277.0,88.0,0.758904
973811685656289740,285.0,80.0,0.780822


In [207]:
df=listings[["id","availability_365", "neighbourhood"]]

In [208]:
df

,id,availability_365,neighbourhood
0,92644,217,NaN
1,93015,40,"Hammersmith, England, United Kingdom"
2,13913,360,"Islington, Greater London, United Kingdom"
3,15400,73,"London, United Kingdom"
4,93734,196,NaN
...,...,...,...
87941,973781286754517228,239,NaN
87942,973801695874775338,88,NaN
87943,973811685656289740,80,NaN
87944,973882998775927897,364,NaN


available,f,t,booked
listing_id,,,
13913,5.0,360.0,0.013699
15400,292.0,73.0,0.800000
17402,65.0,300.0,0.178082
24328,365.0,NaN,1.000000
25123,365.0,NaN,1.000000
...,...,...,...
973781286754517228,126.0,239.0,0.345205
973801695874775338,277.0,88.0,0.758904
973811685656289740,285.0,80.0,0.780822
